## R Notebook for repeated-measures ANOVA lecture

Based on [Crosse et al (2015)](https://www.jneurosci.org/content/35/42/14195.short)

This is an R notebook based on what I used to simulate the data and generate the figures for my lecture on repeated-measures ANOVA. To run the code, click on each block (a "cell") of code and click the "play" button near the top of the page. There is also a button at the top of the page to run all of the cells (the two right arrows, or "fast-forward icon", at the top).

*I originally wrote this script in [Python](https://mybinder.org/v2/gh/natezuk/RM-S2-Stats-Demos/b55130f298fb6f07e55bc51aa58085637571ec0a?urlpath=lab%2Ftree%2Fpsyc20255-rm-s2-statistics-rmanova.ipynb), but I converted it into R with the help of Microsoft Copilot and Claude 3.5 Sonnet.*


In [ ]:
# Load necessary libraries
library(tidyverse)
library(reshape2)
library(afex)
library(emmeans)
library(patchwork)
library(car)

# Increase plot size
options(repr.plot.width=12, repr.plot.height=7)

### Generate the data

In [ ]:
# Set seed for reproducibility
#set.seed(42)

# Number of subjects and levels (conditions)
n_subjects <- 20
levels <- c('AV', 'A', 'V')
n_levels <- length(levels)

# Mean values and standard deviation for each level
means <- c(600, 650, 800)
std_dev_sbj <- 100 # standard deviation across participants
std_dev_wthn <- 30 # standard deviation within participants

# Initialize empty data frame to store results
df <- data.frame(
  Subject = numeric(),
  Condition = character(),
  Response_time = numeric()
)

# Simulate data
for (subject in 1:n_subjects) {
  sbj_avg <- rnorm(1, mean = 0, sd = std_dev_sbj)
  for (level in 1:n_levels) {
    value <- rnorm(1, mean = means[level], sd = std_dev_wthn)
    value <- value + sbj_avg
    
    # Add row to data frame
    df <- rbind(df, data.frame(
      Subject = subject,
      Condition = levels[level],
      Response_time = value
    ))
  }
}

glimpse(df)

### Plot the data

In [ ]:
# Plot the data
ggplot(df, aes(x = Condition, y = Response_time, group = Subject)) +
  geom_line(aes(color = Subject)) +
  geom_point() +
  labs(title = "Response time to target word", x = "Condition", y = "Response time (ms)") +
  theme_grey(base_size=20)

### Run the repeated measures ANOVA

In [ ]:
# Run the repeated-measures ANOVA
anova_results <- aov_car(Response_time ~ Error(Subject/Condition), data=df)
summary_aov <- summary(anova_results)
summary_aov

In [ ]:
# Get and display the main information for the within-subjects effect
univar_anova <- summary_aov$univariate.tests
dof1 <- univar_anova[2,'num Df']
dof2 <- univar_anova[2,'den Df']
Fstat <- univar_anova[2,'F value']
pval <- univar_anova[2,'Pr(>F)']
ges <- anova_results$anova_table[1,'ges']

cat(sprintf("Degrees of freedom = (%.0f,%.0f)\n", dof1, dof2))
cat(sprintf("F-statistic = %.2f\n", Fstat))
cat(sprintf("pval = %.3f\n", pval))
cat(sprintf("ges = %.3f\n", ges))

In [ ]:
# Calculate the between-subjects sum-of-squares
SS_total <- sum((df$Response_time - mean(df$Response_time))^2)
    # This will be different than what you see in the ANOVA table above. 
    # R displays the sum-of-squares without subtracting the mean. 
    # Calculating the total SS centered on the mean is more standard.
sbj_mean <- aggregate(Response_time ~ Subject, df, mean)
SS_sbj <- sum((sbj_mean$Response_time - mean(df$Response_time))^2) * n_levels
SS_within <- SS_total - SS_sbj

cat(sprintf("Total SS = %.2f\n", SS_total))
cat(sprintf("Between-subject SS = %.2f\n", SS_sbj))
cat(sprintf("Within-subject SS = %.2f\n", SS_within))

### Demonstration of calculating the p-value

In [ ]:
# Plot the F-distribution
x <- seq(0, 500, length.out = 1000)
y <- df(x, dof1, dof2)

plot(x, y, type = "l", main = sprintf("F-distribution (%.0f, %.0f)", dof1, dof2), 
     xlab = "F value", ylab = "Probability density", log="y")
abline(v = Fstat, col = "red", lty = 2)
polygon(c(Fstat, x[x >= Fstat]), c(min(y[y>0]), y[x >= Fstat]), col = "gray", border = NA)
legend("topright", legend = sprintf("F-statistic = %.1f", Fstat), col = "red", lty = 2)

### Sphericity

In [ ]:
# Reshape data from long to wide format
paired_cond <- pivot_wider(df, 
                          id_cols = Subject, 
                          names_from = Condition, 
                          values_from = Response_time)

# Create the three scatter plots with regression lines
p1 <- ggplot(paired_cond, aes(x = A, y = V)) +
  geom_point() +
  geom_smooth(method = "lm", se = TRUE) +
  xlim(400, 1000) +
  ylim(400, 1000) +
  theme_gray(base_size=20)

p2 <- ggplot(paired_cond, aes(x = AV, y = V)) +
  geom_point() +
  geom_smooth(method = "lm", se = TRUE) +
  xlim(400, 1000) +
  ylim(400, 1000) +
  theme_gray(base_size=20)

p3 <- ggplot(paired_cond, aes(x = AV, y = A)) +
  geom_point() +
  geom_smooth(method = "lm", se = TRUE) +
  xlim(400, 1000) +
  ylim(400, 1000) +
  theme_gray(base_size=20)

# Combine plots side by side
p1 + p2 + p3

In [ ]:
# Variability in the difference between conditions
diff_cond <- data.frame(
  `V-A` = paired_cond$V - paired_cond$A,
  `V-AV` = paired_cond$V - paired_cond$AV,
  `A-AV` = paired_cond$A - paired_cond$AV
)
print(diff_cond)
# This automatically changes the minus signs to ., - is not allowed here

In [ ]:
# Plot the difference between conditions with a box-whisker plot
ggplot(melt(diff_cond), aes(x = variable, y = value)) +
  geom_boxplot() +
  geom_jitter(width = 0.2) +
  labs(x = "Condition pairs", y = "Difference in response time") +
  theme_gray(base_size=20)

In [ ]:
# Mauchly's test for sphericity
sphericity_test <- summary_aov$sphericity.tests
cat(sprintf("Mauchly's sphericity test: W = %.3f, p = %.3f",
           sphericity_test[1,'Test statistic'],sphericity_test[1,'p-value']))

### Multiple comparisons

In [ ]:
# Multiple comparisons
emmeans_results <- emmeans(anova_results, pairwise ~ Condition, adjust="bonf")
print(emmeans_results$contrasts)